# 多層感知器Dropout實作 模型載入

## 載入套件

In [2]:
import tensorflow as tf
import input_data
tf.set_random_seed(1234)

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 訓練資料

In [3]:
batch_xs, batch_ys = mnist.train.next_batch(5)

## 建立Computational Graph

In [4]:
x_ = tf.placeholder(tf.float32, [None, 784], name="x_")
y_ = tf.placeholder(tf.float32, [None, 10], name="y_")

W1 = tf.Variable(tf.truncated_normal([784,800], stddev=0.1), name="W1")
b1 = tf.Variable(tf.zeros([800]), name="b1")
h1 = tf.nn.sigmoid(tf.matmul(x_, W1) + b1)

keep_prob = tf.placeholder(tf.float32, name="keep_prob")
h1_drop = tf.nn.dropout(h1, keep_prob)

W2 = tf.Variable(tf.truncated_normal([800,10], stddev=0.1), name="W2")
b2 = tf.Variable(tf.zeros([10]), name="b2")
y = tf.nn.softmax(tf.matmul(h1_drop, W2) + b2)

cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

optimizer = tf.train.GradientDescentOptimizer(0.003)
trainer = optimizer.minimize(cross_entropy)

init = tf.initialize_all_variables()


## Session

In [5]:
sess = tf.Session()
sess.run(init)

## Dropout Layer

In [6]:
sess.run(h1_drop, feed_dict={x_: batch_xs, y_: batch_ys, keep_prob:0.5})


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.10676885,  0.        ],
       [ 1.23455167,  1.45929801,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.13515639,  0.51538843,  0.        , ...,  1.59890199,
         0.        ,  0.        ],
       [ 0.        ,  1.28287745,  0.5165872 , ...,  0.        ,
         0.        ,  0.68229574],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

## 建立成效評估用的Computational Graph

In [7]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Saver

In [8]:
saver = tf.train.Saver(max_to_keep=10)


## 模型載入＆模型選擇

In [9]:
keep_probs = [0.4,0.5,0.6,0.7,0.8,0.9]
for k_prob in keep_probs:
    model_name = "model_dropout_%s.ckpt"%( str(k_prob).replace(".","") )
    saver.restore(sess, model_name)

    valid_accurarcy = sess.run(accuracy, feed_dict={x_: mnist.validation.images, y_: mnist.validation.labels, keep_prob:1})
    test_accurarcy = sess.run(accuracy, feed_dict={x_: mnist.test.images, y_: mnist.test.labels, keep_prob:1})
    print "keep_prob:%s, valid:%s, test:%s"%(k_prob, valid_accurarcy,test_accurarcy)

keep_prob:0.4, valid:0.968, test:0.9634
keep_prob:0.5, valid:0.9724, test:0.9672
keep_prob:0.6, valid:0.9742, test:0.9704
keep_prob:0.7, valid:0.9766, test:0.9713
keep_prob:0.8, valid:0.9782, test:0.9754
keep_prob:0.9, valid:0.9792, test:0.9758
